##### 10/15/20:
Created by Noah Francis

### Parser Function

In [ ]:
import numpy
from numpy import array
from datetime import datetime, timezone

# read STK file (rough probably)
def read_STK(filename):
    '''Read and parse an STK ECEF satellite position file (assumes data directory)
    ----------------------------------------------------------------------------
    Inputs:
    --------
    `filename` -- string with the file name
    
    Returns:
    --------
    `data` -- dictionary containing the STK file info
    '''

    data_dir = '/Users/rockm/Documents/CU_Boulder/ASEN_4018/Software/'
    
    # create dictionary
    data = {
        'Time': [],
        'r_vec': [],
        'r_mag': [],
    }
    
    # month dictionary
    month = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
    }
    
    # parse data
    with open(data_dir + filename, 'r') as file:
        lines = list(file.readlines())
    for i, line in enumerate(lines):
        if (i >= 7):
            Line = line.split()

            # Time stuff
            year = int(Line[2])
            day = int(Line[0])
            hour = int(Line[3][0:2])
            minute = int(Line[3][3:5])
            # can add seconds here once we start caring about them

            # Space stuff
            x = float(Line[4]) 
            y = float(Line[5])
            z = float(Line[6])
            r_vec = array([x, y, z])
            r_mag = float(Line[7]) # magnitude of position vector

            # append Time and Space to the data dictionary's lists
            data['Time'].append(datetime(year, month[Line[1]], day, hour, minute, tzinfo=timezone.utc))
            data['r_vec'].append(r_vec)
            data['r_mag'].append(r_mag)
    data['r_vec'] = array(data['r_vec'])
    
    return data

#### Parser Scratch Work 

In [ ]:
# data = {
#     'Time': [],
#     'x': [],
#     'y': [],
#     'z': [],
#     'r': [],
# } 
# data.keys()

month = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

In [ ]:
data_dir = '/Users/rockm/Documents/CU_Boulder/ASEN_4018/Software/'
filename = 'CSIM_ECEF_Position.txt'

# create dictionary
data = {
    'Time': [],
    'r_vec': [],
    'r_mag': [],
}

with open(data_dir + filename, 'r') as file:
    lines = list(file.readlines())
for i, line in enumerate(lines):
    if (i >= 7):
        Line = line.split()
        
        # Time stuff
        year = int(Line[2])
        day = int(Line[0])
        hour = int(Line[3][0:2])
        minute = int(Line[3][3:5])
        # could add seconds here somehow
        
        # Space stuff
        x = float(Line[4]) 
        y = float(Line[5])
        z = float(Line[6])
        r_vec = [x, y, z]
        r_mag = float(Line[7]) # magnitude of position vector

        # append Time and Space to the data dictionary's lists
        data['Time'].append(datetime(year, month[Line[1]], day, hour, minute))
        data['r_vec'].append(r_vec)
        data['r_mag'].append(r_mag)

In [4]:
Line = lines[258].split()
year = int(Line[2])
day = int(Line[0])
hour = int(Line[3][0:2])
minute = int(Line[3][3:5])
# second = float(Line[3][6:12])
print(Line)
# print(hour, minute, second
datetime(year, month[Line[1]], day, hour, minute) 
# map(float, Line[3])

### Gibb's Method Function

In [5]:
import numpy
from numpy import array, cross, dot, sqrt
from numpy.linalg import norm

# Gibb's Method 
def Gibby(r_vec, r_mag, mu):
    '''Use Gibb's Method to produce a velocity vector from three consecutive
    position vectors.
    ----------------------------------------------------------------------------
    Inputs:
    --------
    `r_vec` -- numpy array of three consecutive (in time) position vectors (3,3) [km]
    `r_mag` -- list of corresponding vector magnitudes [km]
    `mu`    -- Standard Gravitational Parameter of the central body [km**3 / s**2]
    
    Returns:
    --------
    `v_vec`        -- velocity vector (numpy array) at the second position vector's 
                      time (1,3) [km/s]
    `coplanar_val` -- dot product of unit vectors that should be coplanar for Gibb's
                      Method to work properly [~] (closer to zero is better)
    '''
    
    # compute unit vectors
    C_01 = cross(r_vec[0], r_vec[1]) / norm(cross(r_vec[0], r_vec[1]))
    C_12 = cross(r_vec[1], r_vec[2]) / norm(cross(r_vec[1], r_vec[2]))
    C_20 = cross(r_vec[2], r_vec[0]) / norm(cross(r_vec[2], r_vec[0]))
    
    # check how coplanar the position vectors are
    coplanar_val = dot((r_vec[0] / r_mag[0]), C_12)
    
    # calculate N, D, and S
    N = r_mag[0] * cross(r_vec[1], r_vec[2]) + r_mag[1] * cross(r_vec[2], r_vec[0]) + r_mag[2] * cross(r_vec[0], r_vec[1])
    D = cross(r_vec[0], r_vec[1]) + cross(r_vec[1], r_vec[2]) + cross(r_vec[2], r_vec[0])
    S = r_vec[0] * (r_mag[1] - r_mag[2]) + r_vec[1] * (r_mag[2] - r_mag[0]) + r_vec[2] * (r_mag[0] - r_mag[1])
    
    # calculate velocity vector at second position vector
    v_vec = sqrt(mu / (norm(N) * norm(D))) * ((cross(D, r_vec[1]) / r_mag[1]) + S)
    
    return v_vec, coplanar_val

#### Gibb's Method Scratch Work

In [6]:
from numpy import cross, array

a = [1,1,1]
b = [5,1,2]
c = norm(cross(a,b))
print(c)

5.0990195135927845


In [7]:
a = [2, 4, 6]
[n / 2 for n in a]

[1.0, 2.0, 3.0]

In [8]:
[array([2,3,4]), array([2,2,2]), array([1,2,5])]

[array([2, 3, 4]), array([2, 2, 2]), array([1, 2, 5])]

In [9]:
a = [0,1,2,3,4,5,6,7,8,9]
a[:6:2]

[0, 2, 4]

### Call Functions

In [2]:
from utilities.useful_functions import ecef2eci

# Standard Gravitational Parameter of Earth
# mu_E = 3.986004418e14 #[m**3 / s**2]
mu_E = 398600.4418 #[km**3 / s**2]

# parse STK text file data
data = read_STK('CSIM_ECEF_Position.txt')

# data['r_vec'] = ecef2eci(data['r_vec'], data['Time'])
data['r_mag'] = norm(data['r_vec'], axis=1)

# use Gibb's method
V, Fred = Gibby(data['r_vec'][0:12000:4000], data['r_mag'][0:12000:4000], mu_E) 
print('Orbital Speed:', norm(V), '[km/s]')
print('Coplanar Value:', Fred)

NameError: name 'read_STK' is not defined

## SpaceNet Matlab Application
---
Matlab app development for CU Boulder Aerospace Engineering Senior Projects Team SpaceNet. 

#### **TLE_DATA_V_1_1.mlapp**
Version 1.1 of the TLE Data app

#### **TLE_DATA_V_3_1.mlapp**
New format includes TLE comparison tab

#### **TLE_DATA_V_3_2.mlapp**
Gibb's method and orbital element calculation functions implemented



In [1]:
from numpy import linspace


In [8]:
linspace(50,500,num=30)

array([ 50.        ,  65.51724138,  81.03448276,  96.55172414,
       112.06896552, 127.5862069 , 143.10344828, 158.62068966,
       174.13793103, 189.65517241, 205.17241379, 220.68965517,
       236.20689655, 251.72413793, 267.24137931, 282.75862069,
       298.27586207, 313.79310345, 329.31034483, 344.82758621,
       360.34482759, 375.86206897, 391.37931034, 406.89655172,
       422.4137931 , 437.93103448, 453.44827586, 468.96551724,
       484.48275862, 500.        ])